## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Komsomolskiy,UZ,40.4272,71.7189,44.02,33,0,2.30,clear sky
1,1,Vaini,TO,-21.2000,-175.2000,77.16,94,100,10.36,light rain
2,2,Pisco,PE,-13.7000,-76.2167,66.25,82,0,4.61,clear sky
3,3,Khovu-Aksy,RU,51.1333,93.6000,2.62,80,21,2.73,few clouds
4,4,Albany,US,42.6001,-73.9662,15.31,85,100,1.99,overcast clouds


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,77.16,94,100,10.36,light rain
11,11,Rikitea,PF,-23.1203,-134.9692,78.49,73,13,7.85,few clouds
14,14,Butaritari,KI,3.0707,172.7902,82.96,78,14,18.19,few clouds
16,16,Bambous Virieux,MU,-20.3428,57.7575,83.21,83,92,19.37,light rain
29,29,Vao,NC,-22.6667,167.4833,80.37,85,99,7.27,overcast clouds
38,38,Bosobolo,CD,4.1833,19.9000,80.15,20,100,2.93,overcast clouds
42,42,Aitape,PG,-3.1374,142.3475,83.21,77,100,9.66,overcast clouds
45,45,Sarangani,PH,5.4033,125.4636,81.57,79,99,26.15,overcast clouds
53,53,Moyo,UG,3.6609,31.7247,84.34,16,100,9.53,overcast clouds
56,56,Kavieng,PG,-2.5744,150.7967,83.66,71,92,8.21,overcast clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,77.16,94,100,10.36,light rain
11,11,Rikitea,PF,-23.1203,-134.9692,78.49,73,13,7.85,few clouds
14,14,Butaritari,KI,3.0707,172.7902,82.96,78,14,18.19,few clouds
16,16,Bambous Virieux,MU,-20.3428,57.7575,83.21,83,92,19.37,light rain
29,29,Vao,NC,-22.6667,167.4833,80.37,85,99,7.27,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
686,686,Broken Hill,AU,-31.9500,141.4333,82.11,24,40,16.11,scattered clouds
696,696,Mildura,AU,-34.2000,142.1500,85.06,20,40,12.66,scattered clouds
702,702,Katherine,AU,-14.4667,132.2667,84.52,65,100,0.00,overcast clouds
703,703,Santa Marta,CO,11.2408,-74.1990,79.90,70,13,8.01,few clouds


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,light rain,-21.2000,-175.2000,
11,Rikitea,PF,78.49,few clouds,-23.1203,-134.9692,
14,Butaritari,KI,82.96,few clouds,3.0707,172.7902,
16,Bambous Virieux,MU,83.21,light rain,-20.3428,57.7575,
29,Vao,NC,80.37,overcast clouds,-22.6667,167.4833,
38,Bosobolo,CD,80.15,overcast clouds,4.1833,19.9000,
42,Aitape,PG,83.21,overcast clouds,-3.1374,142.3475,
45,Sarangani,PH,81.57,overcast clouds,5.4033,125.4636,
53,Moyo,UG,84.34,overcast clouds,3.6609,31.7247,
56,Kavieng,PG,83.66,overcast clouds,-2.5744,150.7967,


In [60]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")           

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [61]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,light rain,-21.2000,-175.2000,Keleti Beach Resort
11,Rikitea,PF,78.49,few clouds,-23.1203,-134.9692,People ThankYou
14,Butaritari,KI,82.96,few clouds,3.0707,172.7902,Isles Sunset Lodge
16,Bambous Virieux,MU,83.21,light rain,-20.3428,57.7575,Casa Tia Villa
29,Vao,NC,80.37,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny


In [66]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,light rain,-21.2000,-175.2000,Keleti Beach Resort
11,Rikitea,PF,78.49,few clouds,-23.1203,-134.9692,People ThankYou
14,Butaritari,KI,82.96,few clouds,3.0707,172.7902,Isles Sunset Lodge
16,Bambous Virieux,MU,83.21,light rain,-20.3428,57.7575,Casa Tia Villa
29,Vao,NC,80.37,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny


In [67]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [68]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [69]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))